# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [8]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [9]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [10]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [11]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_lumpy,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 118.72394779111667 minutes


In [12]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,FOODS_1_005_CA_1_validation,5.0,1.078572,0.1,0.1
1,FOODS_1_005_CA_1_validation,3.0,1.354501,0.1,0.1
2,FOODS_1_005_CA_1_validation,3.0,1.490562,0.1,0.1
3,FOODS_1_005_CA_1_validation,2.0,1.618431,0.1,0.1
4,FOODS_1_005_CA_1_validation,11.0,1.664483,0.1,0.1
...,...,...,...,...,...
3824167,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9,0.9
3824168,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9,0.9
3824169,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9,0.9
3824170,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9,0.9


In [13]:
pattern_df_result_all_params.to_csv('CrostonTSB_Lumpy_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [14]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [15]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 3.880033051549981 minutes


In [16]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_2_validation,0.1,0.1,0.825064,110.7570,125.1519,52.4224,1.466781,1.762906
1,FOODS_1_001_CA_2_validation,0.1,0.2,0.832917,111.8111,126.8607,51.8156,1.480742,1.814954
2,FOODS_1_001_CA_2_validation,0.1,0.3,0.844477,113.3630,128.1891,51.8847,1.501293,1.863622
3,FOODS_1_001_CA_2_validation,0.1,0.4,0.857575,115.1212,130.0971,53.4607,1.524578,1.910206
4,FOODS_1_001_CA_2_validation,0.1,0.5,0.872265,117.0932,132.8041,56.4401,1.550693,1.957295
...,...,...,...,...,...,...,...,...,...
103351,FOODS_3_827_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
103352,FOODS_3_827_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
103353,FOODS_3_827_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
103354,FOODS_3_827_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


# Check what products has unexpected metrics result

In [17]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2187,FOODS_1_017_CA_1_validation,0.1,0.1,inf,inf,200.0,NaN,1.913212e-02,2.724544e-02
2188,FOODS_1_017_CA_1_validation,0.1,0.2,inf,inf,200.0,NaN,1.233952e-03,2.502595e-03
2189,FOODS_1_017_CA_1_validation,0.1,0.3,inf,inf,200.0,NaN,7.122093e-05,1.819891e-04
2190,FOODS_1_017_CA_1_validation,0.1,0.4,inf,inf,200.0,NaN,2.866802e-06,8.719039e-06
2191,FOODS_1_017_CA_1_validation,0.1,0.5,inf,inf,200.0,NaN,6.648501e-08,2.334877e-07
...,...,...,...,...,...,...,...,...,...
103351,FOODS_3_827_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103352,FOODS_3_827_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103353,FOODS_3_827_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103354,FOODS_3_827_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00


In [18]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2187,FOODS_1_017_CA_1_validation,0.1,0.1,inf,inf,200.0,NaN,1.913212e-02,2.724544e-02
2188,FOODS_1_017_CA_1_validation,0.1,0.2,inf,inf,200.0,NaN,1.233952e-03,2.502595e-03
2189,FOODS_1_017_CA_1_validation,0.1,0.3,inf,inf,200.0,NaN,7.122093e-05,1.819891e-04
2190,FOODS_1_017_CA_1_validation,0.1,0.4,inf,inf,200.0,NaN,2.866802e-06,8.719039e-06
2191,FOODS_1_017_CA_1_validation,0.1,0.5,inf,inf,200.0,NaN,6.648501e-08,2.334877e-07
...,...,...,...,...,...,...,...,...,...
103351,FOODS_3_827_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103352,FOODS_3_827_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103353,FOODS_3_827_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103354,FOODS_3_827_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00


In [19]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4536,FOODS_1_039_CA_2_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
4537,FOODS_1_039_CA_2_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
4538,FOODS_1_039_CA_2_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
4539,FOODS_1_039_CA_2_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4540,FOODS_1_039_CA_2_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
103351,FOODS_3_827_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
103352,FOODS_3_827_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
103353,FOODS_3_827_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
103354,FOODS_3_827_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2187,FOODS_1_017_CA_1_validation,0.1,0.1,inf,inf,200.0,NaN,1.913212e-02,2.724544e-02
2188,FOODS_1_017_CA_1_validation,0.1,0.2,inf,inf,200.0,NaN,1.233952e-03,2.502595e-03
2189,FOODS_1_017_CA_1_validation,0.1,0.3,inf,inf,200.0,NaN,7.122093e-05,1.819891e-04
2190,FOODS_1_017_CA_1_validation,0.1,0.4,inf,inf,200.0,NaN,2.866802e-06,8.719039e-06
2191,FOODS_1_017_CA_1_validation,0.1,0.5,inf,inf,200.0,NaN,6.648501e-08,2.334877e-07
...,...,...,...,...,...,...,...,...,...
103351,FOODS_3_827_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103352,FOODS_3_827_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103353,FOODS_3_827_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
103354,FOODS_3_827_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [23]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [24]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 18.887147335423197 %
Percentage of unexpected values of WMAPE is: 18.887147335423197 %
Percentage of unexpected values of SMAPE is: 17.0846394984326 %
Percentage of unexpected values of MAPE is: 18.887147335423197 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [25]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [26]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE       WMAPE       SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                 
0.1   0.1   0.895926  110.858110  124.439638  65.071512  1.721510  2.295767
      0.2   0.883675  108.865078  125.170138  66.623079  1.716193  2.304945
      0.3   0.883056  108.752194  126.192690  68.350243  1.719754  2.322587
      0.4   0.885474  109.071731  127.418082  70.271080  1.727373  2.343988
      0.5   0.890229  109.685042  128.929180  72.558985  1.738984  2.368535
...              ...         ...         ...        ...       ...       ...
0.9   0.5   0.940447  114.667859  134.361796  80.823889  1.920121  2.696016
      0.6   0.946421  115.443933  135.787776  83.022770  1.933673  2.730805
      0.7   0.954139  116.437543  137.519546  85.492450  1.950258  2.769890
      0.8   0.964176  117.708671  139.721312  88.375733  1.971115  2.814127
      0.9   0.976264  119.224065  142.609459  91.705416  1.996337  2.864782

[81 rows x 6 columns]

In [27]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.2, 0.3)
The optimum Alpha and Beta based on Mean WMAPE is: (0.2, 0.3)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [28]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE      WMAPE      SMAPE      MAPE       MAE      RMSE
Alpha Beta                                                              
0.1   0.1   0.830426   97.64705  117.60440  60.22230  1.310746  1.760393
      0.2   0.835663   97.36310  118.84340  61.08360  1.319506  1.774238
      0.3   0.842318   97.61480  120.79915  62.48590  1.334592  1.789417
      0.4   0.847566   97.85375  122.94915  64.63825  1.354353  1.825759
      0.5   0.858541   98.39800  125.76060  67.15950  1.366631  1.851314
...              ...        ...        ...       ...       ...       ...
0.9   0.5   0.928991  107.41075  132.75970  75.12350  1.475186  2.026254
      0.6   0.937872  108.28890  135.16200  78.02930  1.500330  2.067659
      0.7   0.945315  109.64930  137.69180  80.56225  1.511061  2.107575
      0.8   0.955343  111.26050  140.96445  83.61660  1.538367  2.172387
      0.9   0.967308  113.43500  144.57075  87.44735  1.573841  2.238041

[81 rows x 6 columns]

In [29]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of RMSE is: (0.2, 0.1)


# Run Croston model after decide best paramaters

In [30]:
best_alpha = 0.2
best_beta = 0.3

In [31]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [32]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_lumpy, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.2649884849666705 minutes


In [33]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [34]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_2_validation,0.686772,82.6613,93.5813,46.9215,1.003744,1.169911
1,FOODS_1_001_CA_3_validation,1.013776,174.6814,181.4662,66.3934,1.247724,1.954160
2,FOODS_1_002_CA_2_validation,0.759562,136.3317,149.1768,57.0093,0.584279,0.594230
3,FOODS_1_003_CA_2_validation,1.004841,76.9520,88.2974,72.9864,2.473456,3.138016
4,FOODS_1_003_CA_3_validation,0.983977,136.2430,149.6729,57.5570,0.681215,0.719897
...,...,...,...,...,...,...,...
1271,FOODS_3_824_CA_1_validation,2.579054,555.4885,192.8526,66.6384,0.396777,0.500114
1272,FOODS_3_824_CA_2_validation,1.812958,390.4832,198.7903,95.5790,0.278917,0.402691
1273,FOODS_3_827_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
1274,FOODS_3_827_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000


In [35]:
df_result_final.to_csv('CrostonTSB_Lumpy_Test_Data.csv')